In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import  pandas as pd 
import os

# file_path = os.path.join(os.getcwd(), 'app', 'data', 'zehrs', 'zehrs_data.csv')
store_data = pd.read_csv('../app/data/zehrs/zehrs_data.csv')

items = store_data['product'].unique()
vectorizer = TfidfVectorizer(min_df=1)
tf_idf_matrix = vectorizer.fit_transform(items)


In [6]:
import numpy as np
from scipy.sparse import csr_matrix
# ! pip install sparse_dot_topn 
import sparse_dot_topn.sparse_dot_topn as ct

  Using cached sparse_dot_topn-0.3.4-cp38-cp38-macosx_10_9_x86_64.whl (316 kB)


In [7]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)
    
    ct.sparse_dot_topn(
            M, N, np.asarray(A.indptr, dtype=idx_dtype),
            np.asarray(A.indices, dtype=idx_dtype),
            A.data,
            np.asarray(B.indptr, dtype=idx_dtype),
            np.asarray(B.indices, dtype=idx_dtype),
            B.data,
            ntop,
            lower_bound,
            indptr, indices, data)
            
    return csr_matrix((data,indices,indptr),shape=(M,N))


In [8]:
import time
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.85)
t = time.time()-t1
print("SELFTIMED:", t)


SELFTIMED: 0.024100303649902344


In [12]:
matches

<1x6985 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
import  pandas as pd 
import os

# file_path = os.path.join(os.getcwd(), 'app', 'data', 'zehrs', 'zehrs_data.csv')
store_data = pd.read_csv('../app/data/zehrs/zehrs_data.csv')

items = store_data['product'].unique()
vectorizer = TfidfVectorizer(min_df=1)
tf_idf_matrix = vectorizer.fit_transform(items)


In [19]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity('apples', 'oranges')

ValueError: could not convert string to float: 'apples'

In [34]:
import math
import re
from collections import Counter

WORD = re.compile(r"\w+")


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

text1 = "frozen pizza"
text2 = "Thin & Crispy Pepperoni Pizza"

vector1 = text_to_vector(text1.lower())
vector2 = text_to_vector(text2.lower() + f' {department}')

cosine = get_cosine(vector1, vector2)

print("Cosine:", cosine)


Cosine: 0.6324555320336759


In [33]:
text2.lower() + department

'thin & crispy pepperoni pizzafrozen'

In [36]:
store_data.loc[store_data['category'] == 'frozen']

,store,category,brand,product,full_product_text,price,sale_price,price_unit,per_unit_price,sale_per_unit_price,units,price_per_1,is_sale,data_last_refreshed_at,list_price,sale_valid_until
7160,zehrs,frozen,President's Choice,Sliced Strawberries,President's Choice Sliced Strawberries 600 g,5.99,NaN,each,1.000000,NaN,100g,True,False,02/23/2023,5.99,NaN
7161,zehrs,frozen,President's Choice,Broccoli Florets,President's Choice Broccoli Florets 500 g,4.00,3.49,each,0.802292,0.70,100g,True,True,02/23/2023,3.49,03/01
7162,zehrs,frozen,President's Choice,Blueberries Cultivated,President's Choice Blueberries Cultivated 600 g,5.99,NaN,each,1.000000,NaN,100g,True,False,02/23/2023,5.99,NaN
7163,zehrs,frozen,CAVENDISH FARMS,Flavourcrisp Classic Straight Cut Fries,CAVENDISH FARMS Flavourcrisp Classic Straight...,4.99,NaN,each,0.670000,NaN,100g,True,False,02/23/2023,4.99,NaN
7164,zehrs,frozen,Mccain,Superfries 5 Minute Shoestring Fries,Mccain Superfries 5 Minute Shoestring Fries 650 g,4.99,NaN,each,0.770000,NaN,100g,True,False,02/23/2023,4.99,NaN
7165,zehrs,frozen,President's Choice,Mango Chunks,President's Choice Mango Chunks 6 00 g,5.99,NaN,each,1.000000,NaN,100g,True,False,02/23/2023,5.99,NaN
7166,zehrs,frozen,No Name,Crinkle Cut Fried Potatoes,No Name Crinkle Cut Fried Potatoes 1 kg,4.49,NaN,each,0.450000,NaN,100g,True,False,02/23/2023,4.49,NaN
7167,zehrs,frozen,CAVENDISH FARMS,Original Hash Brown Patties,CAVENDISH FARMS Original Hash Brown Patties 600 g,4.49,NaN,each,0.750000,NaN,100g,True,False,02/23/2023,4.49,NaN
7168,zehrs,frozen,President's Choice,"Flatbread Chicken, Bacon & Caramelized Onion","President's Choice Flatbread Chicken, Bacon & ...",5.79,NaN,each,1.700000,NaN,100g,True,False,02/23/2023,5.79,NaN
7169,zehrs,frozen,Mccain,Straight Cut French Fries,Mccain Straight Cut French Fries 900 g,4.29,NaN,each,0.480000,NaN,100g,True,False,02/23/2023,4.29,NaN


In [39]:
'frozen pizza'.replace('frozen', '').strip()

'pizza'

In [58]:
item = 'apple'

product = 'apple pie'.split(' ')

(product.index(item)+1) / len(product)

0.5

In [59]:
item = 'apple'

product2 = 'honey crisp apple'.split(' ')

product2.index(item)+1, len(product2), (product2.index(item)+1) / len(product2)

(3, 3, 1.0)

In [60]:
item = 'apple'

product2 = 'cinnamon apple cider'.split(' ')

product2.index(item)+1, len(product2), (product2.index(item)+1) / len(product2)

(2, 3, 0.6666666666666666)

In [95]:
len('Dill Pickle Sour Cream Dip'.split(' '))

5

In [93]:
product = 'Dill Pickle Sour Cream Dip'.lower().replace(',', '')

item = 'sour cream'

split_list = product.split(item)

try:    
    idx = split_list.index('')
except:
    idx = product.split()

split_list = [word for l in split_list for word in l.strip().split()]


split_list.insert(idx, item)
split_list

['dill', 'sour cream', 'pickle', 'dip']

In [86]:
def get_rel(product, item):
    product = ps.stem(product.lower()).replace(',', '')
    split_list = product.split(item)
    idx = split_list.index('')
    split_list = [word for l in split_list for word in l.strip().split()]
    split_list.insert(idx, item)
    
    rel = (split_list.index(item)+1) / len(product[0].split(' '))
    return rel 

In [90]:

from nltk.stem import PorterStemmer
ps = PorterStemmer()

item = 'sour cream'
product = [('Sour Cream', 100, 218), ('Sour Cream', 100, 285), ('Sour Cream, Regular 14%', 90, 5), ('Sour Cream Dip, French Onion', 90, 13), ('Sour Cream, Light 5%', 90, 28), ('Sour Cream, Light 5%', 90, 96), ('Sour Cream, Fat Free 0%', 90, 146), ('Dill Pickle Sour Cream Dip', 90, 219), ('Sour Cream, Fat Free 1%', 90, 253), ('Dill Pickle Sour Cream Dip', 90, 255)]
    

for prod in product: 
    print(prod[0])
    rel = get_rel(prod[0], item)
    

Sour Cream
Sour Cream
Sour Cream, Regular 14%
Sour Cream Dip, French Onion
Sour Cream, Light 5%
Sour Cream, Light 5%
Sour Cream, Fat Free 0%
Dill Pickle Sour Cream Dip


ValueError: '' is not in list

In [102]:
def get_rel(product, item):
    product = ps.stem(product.lower()).replace(',', '')
    split_list = product.split(item)
    idx = split_list.index('')
    split_list = [word for l in split_list for word in l.strip().split()]
    split_list.insert(idx, item)
    
    rel = (split_list.index(item)+1) / len(product[0].split(' '))
    return rel 

In [127]:
item = 'sour cream'
cat_item = item.replace(' ', '-')

products = ['dill pickle sour cream dip', 'sour cream', 'sour cream & onion chips']

for prod in products:
    prod = prod.replace(item, cat_item)

    prod_list = prod.split(' ')

    print(prod, (prod_list.index(cat_item)+1) / len(prod_list))
    


dill pickle sour-cream dip 0.75
sour-cream 1.0
sour-cream & onion chips 0.25


In [115]:
# product = 'dill Sour Cream, Regular 14%'
item = 'pizza'

product = [('Casa di Mama Pepperoni Pizza', 90, 10), ('Thin Crispy Crust Pepperoni Pizza', 90, 36), ('Giuseppe Pizzeria Rising Crust Hawaiian Pizza ', 60, 31), ('Sweet Pineapple Chicken Teriyaki Bowl', 54, 29), ('Samosa Spinach Paneer', 54, 32), ('Ice Cream Premium Pistachio & Almonds ', 54, 34), ('Baby Brussels Sprouts, Club Size', 40, 25), ('Low Fat Frozen Whipped Topping', 40, 42), ('Blueberries Cultivated', 36, 2), (' Flavourcrisp Classic Straight Cut Fries', 36, 3)]

for prod in product:
    print(prod[0])
    rel = 0
    for word in item.split(' '):
        # print((ps.stem(product).lower().replace(',', '').split(' ').index(word)+1) / len(product.split(' ')))

        rel += (ps.stem(prod[0]).lower().replace(',', '').split(' ').index(word)+1) / len(prod[0].split(' '))

    rel = rel / len(item.split(' '))

    print(rel)

Casa di Mama Pepperoni Pizza
1.0
Thin Crispy Crust Pepperoni Pizza
1.0
Giuseppe Pizzeria Rising Crust Hawaiian Pizza 
0.8571428571428571
Sweet Pineapple Chicken Teriyaki Bowl


ValueError: 'pizza' is not in list

In [119]:
(1-0.5+0.1)*2.99, (1-0.9+0.2)*4.99 , (1-0.7+0.2)*1.99

(2.093, 1.497, 0.995)

In [120]:
2.99-0.5, 4.99-0.9, 1.99-0.7

(2.49, 4.09, 1.29)